In [46]:
import numpy as np
import math
import colorsys
from scipy import ndimage
from scipy import misc
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
import json

In [47]:
#converts 1d array into a 2d array where rows are w long
def arr_reshape(arr,w):
    l = len(arr)
    p = math.ceil(l/w)
    out = []
    for i in range(p):
        out.append(arr[i*w:(i+1)*w])
    return out

#return a,b,c components we use for reconstruction
def svd_recon_components(arr,num_eigenvecs,a_decimals,b_decimals,c_decimals):
    u, s, v = np.linalg.svd(arr)
    return [np.round(u[:, :n_vec],decimals=a_decimals),np.round(s[:n_vec],decimals=b_decimals),np.round(v[:n_vec, :],decimals=c_decimals)]

#runs kmeans on a matrix and creates a kstring
def kmeans_serialize_clusters(arr,n_clusts,num_decimals):
    #changing 1d array to 2d array in order run kmeans instead of using some sort of binning algorithm
    sample = list(zip(shuffle(arr.flatten(), random_state=0)[:1500],[0]*1500))
    kmeans = KMeans(n_clusters=n_clusts, random_state=0).fit(sample)
    clusters = np.round(kmeans.cluster_centers_,decimals=num_decimals)[:,0]
    k= list(map(lambda x: kmeans.predict(list(map(lambda y: [y,0],x))),arr))
    k_string = ''.join(list(map(lambda x: ''.join(list(map(chr,x))),k)))
    return clusters.tolist(),k_string

#changes kstrings back to into component matrix. THIS WOULD ESSENTIALLY RUN IN THE BROWSER
def kmeans_unserializer(kstring,clusters,w):
    t= []
    for k in kstring:
        t.append(clusters[ord(k)])
    return arr_reshape(t,w)

In [48]:
#Declaring global vars
h_eigenvecs,s_eigenvecs,v_eigenvecs = (512,512,512) #number of eigenvectors to use for hue,saturation,value (respectively)
decimals_a,decimals_b,decimals_c =(4,2,5) #number of decimal places to use for a,b,c reconstruction components
kstring_clusters =64
img_name ='image5.jpg'
num_color_clusters = 256
nth_col = 7 #remove every nth col #make sure image width is not divisible by number reason:reeeeeeeeeee

In [49]:
%%time
#Read in rgb image. Tested and worked on TIFF and jpg.
img = misc.imread('images/'+img_name).astype(float)
h, w, d = original_shape = tuple(img.shape)
print(original_shape)
assert d == 3 #Basically we want to make sure we have an rgb image not a rgba one

#Doing a double map with colorsys.rgb_to_hsv is slow ;__; so we'll kmeans and then predict
img_arr_sample = shuffle(np.reshape(img, (w * h, d)), random_state=0)[:1000]
color_kmeans = KMeans(n_clusters=num_color_clusters, random_state=0).fit(img_arr_sample)
color_clusters = np.array(list(map(lambda x: colorsys.rgb_to_hsv(x[0]/255,x[1]/255,x[2]/255),color_kmeans.cluster_centers_)))

#use predict to get hsv vals and then reshape in order to column extract hsv
hsv_arr = np.reshape(np.array(list(map(lambda x: color_clusters[x],list(map(color_kmeans.predict,img))))), (w * h, d))

#column extract hue,sat,val arrays and reshape
h_arr = arr_reshape(hsv_arr[:,0],w)
s_arr = arr_reshape(hsv_arr[:,1],w)
v_arr = arr_reshape(hsv_arr[:,2],w)

#time to remove columns

(720, 1280, 3)
CPU times: user 5.28 s, sys: 562 ms, total: 5.84 s
Wall time: 5.97 s


In [43]:
#remove every nth column
def remove_col(arr,n_col):
    res = []
    r_max = len(arr)
    c_max = len(arr[0])
    for i in range(r_max):
        for j in range(c_max):
            if (j)

array([   0.611678 ,    0.3277592,  224.25     ])